# Project 4 - Background Work

## Setting Up

### imports

In [70]:
import os
import pandas as pd
import petl as etl

### create data directory

In [71]:
raw_data_path = "data/raw"
raw_data_directory = []

for filename in os.listdir(raw_data_path):
    file = os.path.join(raw_data_path, filename)

    if os.path.isfile(file):
        #print(file)
        raw_data_directory.append(file)

In [72]:
#raw_data_directory

### remove csv duplicates

In [73]:
len(raw_data_directory)

94

In [74]:
for file in raw_data_directory:
    if str(file).endswith('json'):
        if str(file).replace('json', 'csv') in raw_data_directory:
            raw_data_directory.remove(str(file).replace('json', 'csv'))

len(raw_data_directory)

86

In [75]:
#raw_data_directory

### acquire product list

In [76]:
# acquire product list from directory
products_table = etl.fromcsv('data/other/products_list.csv')
# cut wanted fields
products_table = etl.cut(products_table, 'product', 'category')

products_table = etl.values(products_table, 'product', 'category')

product_list = list(products_table)

# print for checking
product_list


[('Fairtrade Bananas Loose', 'fruits & vegetables'),
 ('British carrots loose', 'fruits & vegetables'),
 ('Onions Loose', 'fruits & vegetables'),
 ('British baking potatoes loose', 'fruits & vegetables'),
 ('Red pepper', 'fruits & vegetables'),
 ('Mixed pepper', 'fruits & vegetables'),
 ('Brocolli loose', 'fruits & vegetables'),
 ('Lemon', 'fruits & vegetables'),
 ('Spring onions bunch', 'fruits & vegetables'),
 ('Sweet potatoes loose', 'fruits & vegetables'),
 ('courgette loose', 'fruits & vegetables'),
 ('baby potatoes', 'fruits & vegetables'),
 ('british parsnips loose', 'fruits & vegetables'),
 ('fine beans', 'fruits & vegetables'),
 ('garlic', 'fruits & vegetables'),
 ('celery', 'fruits & vegetables'),
 ('aubergine', 'fruits & vegetables'),
 ('raspberries', 'fruits & vegetables'),
 ('british bramley cooking apples', 'fruits & vegetables'),
 ('easy peeler loose', 'fruits & vegetables'),
 ('scottish salmon', 'meat & fish'),
 ('beef mince', 'meat & fish'),
 ('british fresh chicken br

### creation of blank dfs

In [77]:
best_worst_products_df = pd.DataFrame()

best_worst_products_df

""


## Cleanup & Transform Functions

### adding product category

In [78]:
# add product category function
def add_prod_cat(prv, cur, nxt):
    for row in product_list:
        if cur.product == row[0]:
            return row[1]

### cleanup

In [79]:
# cleanup function
def table_cleanup(table, file):
    
# # remove unwanted fields
# current_table = etl.cutout(
#                     current_table, 
#                     'month', 'day', 'hour')

# convert field to float
# round up field to 2 decimal points
    table = etl.convert(
                    table, 
                    'amount_in_gbp',
                    lambda cell: round(float(cell), 2))

# if table has the header sku or item.. change to product
    if 'sku' in etl.header(table):
        table = etl.rename(
                        table, 
                        'sku', 'product')
    elif 'item' in etl.header(table):
        table = etl.rename(
                        table, 
                        'item', 'product')

# rename header to only have the one name (total_quantity_purchase)
    if 'quantity' in etl.header(table):
        table = etl.rename(
                        table,
                        'quantity', 'total_quantity_purchased'
    )
    elif 'total_quantity' in etl.header(table):
        table = etl.rename(
                        table,
                        'total_quantity', 'total_quantity_purchased'
    )
    elif 'quantity_purchased' in etl.header(table):
        table = etl.rename(
                        table,
                        'quantity_purchased', 'total_quantity_purchased'
    )

# convert field to int
    table = etl.convert(
                    table, 
                    ['total_quantity_purchased'],
                    int)

# adding product categories to table
    table = etl.addfieldusingcontext(table, 'product_category', add_prod_cat)

    # obtain city name
    city_name = str(file).split('_', 1).pop(1).rsplit('.').pop(0)
    # adding city as a field
    table = etl.addfield(table, 'city', city_name)

# return the newly cleaned table
    return table

### df conversion

In [80]:
# df conversion & manipulation func
def something(table):
# convert table to df
    df = etl.todataframe(table)
# group df by product & product category
# obtain the sum of products sold & the amount in gbp
# sort by descending
    groupby_prod_prodcat_df = df.groupby(
                                    ['product', 'product_category', 'city'])\
                                    .sum().sort_values('total_quantity_purchased', ascending=False)
# obtain the best and worst 5 products
    groupby_prod_prodcat_df_ends = groupby_prod_prodcat_df.head().append(groupby_prod_prodcat_df.tail())

# return the newly made df
    return groupby_prod_prodcat_df_ends

## Iterating Through Files

In [89]:
tally = 0
# for each file in the directory
for file in raw_data_directory:
        tally = tally + 1
# check if csv file type
        if str(file).endswith('csv'):
# extract from said type
                current_table = etl.fromcsv(file)
# initiate cleanup functions
                current_table = table_cleanup(current_table, file)
# initiate df conversion & manipulation
                current_df = something(current_table)
# append the cities product data to the total df for cities
                best_worst_products_df = best_worst_products_df.append(current_df)
                if tally == 2:
                        break
# in case of other file type
        else:
# extract from json file type
                current_table = etl.fromjson(file)
# initiate cleanup functions
                current_table = table_cleanup(current_table, file)
# initiate df conversion & manipulation
                current_df = something(current_table)
# append the cities product data to the total df for cities
                best_worst_products_df = best_worst_products_df.append(current_df)

best_worst_products_df




,,,total_quantity_purchased,amount_in_gbp
product,product_category,city,,
almond croisant,bakery,Bassetlaw_outlet,1100943.0,990547.68
minecraft game - xbox,gaming,Bassetlaw_outlet,1084051.0,9762521.80
millionaire square,bakery,Bassetlaw_outlet,1083880.0,1953165.88
garlic,fruits & vegetables,Bassetlaw_outlet,1082691.0,243892.47
organic soured cream,dairy,Bassetlaw_outlet,1081196.0,972273.42
Almond milk,dairy,Bassetlaw_outlet,1033695.0,1766668.33
free range large eggs x12,dairy,Bassetlaw_outlet,1030080.0,1390241.10
Organic mILK,dairy,Bassetlaw_outlet,1029402.0,1389769.11
jam doughnut,bakery,Bassetlaw_outlet,1028871.0,925829.98


## space

In [88]:
test_0 = etl.fromcsv(raw_data_directory[0])
test_1 = etl.fromcsv(raw_data_directory[1])

test_0 = table_cleanup(test_0, raw_data_directory[0])
test_1 = table_cleanup(test_1, raw_data_directory[1])

#test_0
#test_1

test_0 = something(test_0)
test_1 = something(test_1)

test_combine = pd.DataFrame().append(test_0).append(test_1)

test_combine

,,,total_quantity_purchased,amount_in_gbp
product,product_category,city,,
almond croisant,bakery,Bassetlaw_outlet,1100943.0,990547.68
minecraft game - xbox,gaming,Bassetlaw_outlet,1084051.0,9762521.80
millionaire square,bakery,Bassetlaw_outlet,1083880.0,1953165.88
garlic,fruits & vegetables,Bassetlaw_outlet,1082691.0,243892.47
organic soured cream,dairy,Bassetlaw_outlet,1081196.0,972273.42
Almond milk,dairy,Bassetlaw_outlet,1033695.0,1766668.33
free range large eggs x12,dairy,Bassetlaw_outlet,1030080.0,1390241.10
Organic mILK,dairy,Bassetlaw_outlet,1029402.0,1389769.11
jam doughnut,bakery,Bassetlaw_outlet,1028871.0,925829.98


In [92]:
#file_name = str(raw_data_directory[0]).split('\\').pop(1)

best_worst_products_df.to_csv('data/refined/proof.csv')
best_worst_products_df

,,,total_quantity_purchased,amount_in_gbp
product,product_category,city,,
almond croisant,bakery,Bassetlaw_outlet,1100943.0,990547.68
minecraft game - xbox,gaming,Bassetlaw_outlet,1084051.0,9762521.80
millionaire square,bakery,Bassetlaw_outlet,1083880.0,1953165.88
garlic,fruits & vegetables,Bassetlaw_outlet,1082691.0,243892.47
organic soured cream,dairy,Bassetlaw_outlet,1081196.0,972273.42
Almond milk,dairy,Bassetlaw_outlet,1033695.0,1766668.33
free range large eggs x12,dairy,Bassetlaw_outlet,1030080.0,1390241.10
Organic mILK,dairy,Bassetlaw_outlet,1029402.0,1389769.11
jam doughnut,bakery,Bassetlaw_outlet,1028871.0,925829.98
